In [19]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

# Load the Excel file to check its structure
file_path = 'IT DIRECTORATE BIDS  FOR 2023.xlsx'
xls = pd.ExcelFile(file_path)

# Display sheet names to understand its structure
xls.sheet_names


['Main Bids 2018',
 'Sheet1',
 'GL 2200600002',
 'GL 2201900002',
 'GL 2202000002',
 'GL 3112210001',
 'GL 2200600003',
 'GL 2202000004',
 ' HRMS (3112210001)']

In [10]:
# Load data from Sheet1 to inspect its structure
sheet1_data = pd.read_excel(file_path, sheet_name='Sheet1')

# Display the first few rows to understand the data
sheet1_data.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,SUMMARY,NaN,NaN,ZWL,NaN,NaN,USD
2,1.0,NaN,GL ACC,2.200600e+09,Consumables,311083500,NaN,NaN,2222025
3,2.0,NaN,GL ACC,3.112210e+09,ICT Equipment,168230000,NaN,NaN,1201642.86
4,3.0,NaN,GL ACC,2.201900e+09,Part Technical equipment,22629200,NaN,NaN,163208.57


In [11]:
# Clean and preprocess the data by skipping initial irrelevant rows
sheet1_cleaned = pd.read_excel(file_path, sheet_name='Sheet1', skiprows=2)

# Inspect the cleaned data to identify numeric fields for variations
sheet1_cleaned.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 6', 'Unnamed: 7'], inplace=True)
sheet1_cleaned.head()


,SUMMARY,Unnamed: 3,Unnamed: 4,ZWL,USD
0,GL ACC,2.200600e+09,Consumables,311083500.0,2222025.00
1,GL ACC,3.112210e+09,ICT Equipment,168230000.0,1201642.86
2,GL ACC,2.201900e+09,Part Technical equipment,22629200.0,163208.57
3,GL ACC,2.202000e+09,Technical Equipment Maintenance,30150000.0,215357.14
4,GL ACC,2.200600e+09,Renewal of Computer software,523046000.0,3736042.86


In [12]:
import numpy as np

# Define a function to generate synthetic variations
def generate_variations(df, num_variations=30, columns_to_modify=None):
    variations = []
    for _ in range(num_variations):
        variation = df.copy()
        for col in columns_to_modify:
            if col in df.columns:
                variation[col] = df[col] * np.random.uniform(0.8, 1.2, size=len(df))
        variations.append(variation)
    return variations

# Columns to apply variations on
columns_to_modify = ['ZWL', 'USD ']

# Generate 120 variations of the dataset
variations = generate_variations(sheet1_cleaned, num_variations=120, columns_to_modify=columns_to_modify)

# Save all variations to a new Excel file
output_file_path = 'Synthetic_Sheet1_Variations.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    for i, variation in enumerate(variations):
        variation.to_excel(writer, sheet_name=f'Variation_{i+1}', index=False)

output_file_path


'Synthetic_Sheet1_Variations.xlsx'

In [14]:
import pandas as pd

# Load the workbook
file_path = 'Synthetic_Sheet1_Variations.xlsx'
xls = pd.ExcelFile(file_path)

# Initialize an empty list to hold the DataFrames
dfs = []

# Iterate through each sheet and append the data to the list
for sheet_name in xls.sheet_names:
    sheet_df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Extract the integer x from the sheet name
    x = int(sheet_name.split('_')[1])
    
    # Calculate the year based on x
    year = 1995 + (x - 1)
    
    # Create a date range for the entire year
    date_range = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D')
    
    # Assign dates to the rows in the sheet
    sheet_df['Date'] = date_range[:len(sheet_df)]
    
    # Append the DataFrame to the list
    dfs.append(sheet_df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Sort the DataFrame by date
combined_df = combined_df.sort_values(by='Date')

# Save the transformed data to a new file
combined_df.to_csv('transformed_time_series_data.csv', index=False)

print("Transformation complete. Data saved to 'transformed_time_series_data.csv'.")

Transformation complete. Data saved to 'transformed_time_series_data.csv'.


In [16]:
import pandas as pd

# Load the transformed time series data
file_path = 'transformed_time_series_data.csv'
df = pd.read_csv(file_path)

# List of IDs to filter
ids_to_filter = [
    2200600002.0, 3112210001.0, 2201900002.0, 
    2202000002.0, 2200600003.0, 2202000004.0
]

# Filter the DataFrame for each ID and save to a separate CSV file
for id_value in ids_to_filter:
    filtered_df = df[df['Unnamed: 3'] == id_value]
    filtered_df.to_csv(f'time_series_{int(id_value)}.csv', index=False)

print("Time series data frames created and saved.")

Time series data frames created and saved.
